In [66]:
def clean_data(df):

    import pandas as pd
    import datetime as dt
    import numpy as np
   
    # rename columns to get rid of the funny 'EXITS     ' issue
    df.columns = ['C/A','UNIT','SCP','STATION','LINENAME',\
                  'DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
    
    # remove index 180052 (project technology error)
    df.drop(df.index[180052])
    
    # convert to datetime/make turnstile column
    df['converted_time'] = pd.to_datetime(df['DATE']+' '+df['TIME'])
    df['turnstiles'] = df['C/A'] + '-' + df['UNIT'] + '-' + df['SCP'] + '-' + df['STATION']
    
    # sort by date and location
    df_sorted = df.sort_values(['turnstiles', 'converted_time'])
    
    # group by turnstile so we can get entry/exit differences
    turnstile_df = df_sorted.groupby('turnstiles')
    df_sorted['entries_diff'] = turnstile_df['ENTRIES'].diff()
    df_sorted['exits_diff'] = turnstile_df['EXITS'].diff()
    
    # calculates IQR for entries_diff
    Q3 = df_sorted['entries_diff'].quantile(0.75) 
    Q1 = df_sorted['entries_diff'].quantile(0.25)
    IQR = Q3 - Q1
    
    # calculates IQR range using outliers 
    IQR_range = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    
    # removes values outside of lower and upper bounds
    df_sorted = df_sorted[df_sorted['entries_diff'].between(0, IQR_range[1])]
    
    # repeats process for exits_diff
    Q3_2 = df_sorted['exits_diff'].quantile(0.75) 
    Q1_2 = df_sorted['exits_diff'].quantile(0.25)
    IQR_2 = Q3_2 - Q1_2
    IQR_range_2 = (Q1_2 - 1.5 * IQR, Q3_2 + 1.5 * IQR)
    df_sorted = df_sorted[df_sorted['exits_diff'].between(0, IQR_range_2[1])]

    '''
    
    # removes negative values (ONLY NEEDED if no outliers removed)
    df_sorted = df_sorted[df_sorted['entries_diff'].between(0, np.inf)]
    df_sorted = df_sorted[df_sorted['exits_diff'].between(0, np.inf)]
    
    '''
    
    # created new column turnstile_turns with total turnstile interactions per turnstile
    df_sorted['turnstile_turns'] = df_sorted.entries_diff + df_sorted.exits_diff
    df_sorted.turnstile_turns.describe()

    # replaces NaN values with mean for entries_diff and exits_diff
    df_sorted.entries_diff = df_sorted.entries_diff.fillna(df_sorted.entries_diff.mean())
    df_sorted.exits_diff = df_sorted.exits_diff.fillna(df_sorted.exits_diff.mean())

    # provides column day_of_week that designates the day of the week 
    df_sorted['day_of_week']=df_sorted.converted_time.dt.dayofweek
    
    #Create new column to differentiate stations serving different subway lines but with identical names
    df_sorted['station_unique'] = df_sorted['STATION'] + '-' + df_sorted['LINENAME']
    
    return df_sorted

In [5]:
def clean_data_no_outliers(df2):

    import pandas as pd
    import datetime as dt
    import numpy as np
   
    # rename columns to get rid of the funny 'EXITS     ' issue
    df2.columns = ['C/A','UNIT','SCP','STATION','LINENAME',\
                  'DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
    
    
    
    # convert to datetime/make turnstile column
    df2['converted_time'] = pd.to_datetime(df2['DATE']+' '+df2['TIME'])
    df2['turnstiles'] = df2['C/A'] + '-' + df2['UNIT'] + '-' + df2['SCP'] + '-' + df2['STATION']
    
    # sort by date and location
    df2_sorted = df2.sort_values(['turnstiles', 'converted_time'])
    
    # group by turnstile so we can get entry/exit differences
    turnstile2_df = df2_sorted.groupby('turnstiles')
    df2_sorted['entries_diff'] = turnstile2_df['ENTRIES'].diff()
    df2_sorted['exits_diff'] = turnstile2_df['EXITS'].diff()
    
    
    '''
    # calculates IQR for entries_diff
    Q3 = df_sorted['entries_diff'].quantile(0.75) 
    Q1 = df_sorted['entries_diff'].quantile(0.25)
    IQR = Q3 - Q1
    
    # calculates IQR range using outliers 
    IQR_range = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    
    # removes values outside of lower and upper bounds
    df_sorted = df_sorted[df_sorted['entries_diff'].between(0, IQR_range[1])]
    
    # repeats process for exits_diff
    Q3_2 = df_sorted['exits_diff'].quantile(0.75) 
    Q1_2 = df_sorted['exits_diff'].quantile(0.25)
    IQR_2 = Q3_2 - Q1_2
    IQR_range_2 = (Q1_2 - 1.5 * IQR, Q3_2 + 1.5 * IQR)
    df_sorted = df_sorted[df_sorted['exits_diff'].between(0, IQR_range_2[1])]

    '''
    
    # removes negative values (ONLY NEEDED if no outliers removed)
    df2_sorted = df2_sorted[df2_sorted['entries_diff'].between(0, np.inf)]
    df2_sorted = df2_sorted[df2_sorted['exits_diff'].between(0, np.inf)]
    
    # created new column turnstile_turns with total turnstile interactions per turnstile
    df2_sorted['turnstile_turns'] = df2_sorted.entries_diff + df2_sorted.exits_diff
    df2_sorted.turnstile_turns.describe()

    # replaces NaN values with mean for entries_diff and exits_diff
    df2_sorted.entries_diff = df2_sorted.entries_diff.fillna(df2_sorted.entries_diff.mean())
    df2_sorted.exits_diff = df2_sorted.exits_diff.fillna(df2_sorted.exits_diff.mean())

    # provides column day_of_week that designates the day of the week 
    df2_sorted['day_of_week']=df2_sorted.converted_time.dt.dayofweek
    
    return df2_sorted

In [67]:
sf = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190803.txt')
clean_data(sf)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,converted_time,turnstiles,entries_diff,exits_diff,turnstile_turns,day_of_week
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,04:00:00,REGULAR,7148306,2418939,2019-07-27 04:00:00,A002-R051-02-00-00-59 ST,18.0,4.0,22.0,5
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,08:00:00,REGULAR,7148326,2418975,2019-07-27 08:00:00,A002-R051-02-00-00-59 ST,20.0,36.0,56.0,5
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,12:00:00,REGULAR,7148441,2419042,2019-07-27 12:00:00,A002-R051-02-00-00-59 ST,115.0,67.0,182.0,5
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,16:00:00,REGULAR,7148647,2419101,2019-07-27 16:00:00,A002-R051-02-00-00-59 ST,206.0,59.0,265.0,5
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/27/2019,20:00:00,REGULAR,7148908,2419145,2019-07-27 20:00:00,A002-R051-02-00-00-59 ST,261.0,44.0,305.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206991,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/02/2019,05:00:00,REGULAR,5554,386,2019-08-02 05:00:00,TRAM2-R469-00-05-01-RIT-ROOSEVELT,0.0,0.0,0.0,4
206992,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/02/2019,09:00:00,REGULAR,5554,386,2019-08-02 09:00:00,TRAM2-R469-00-05-01-RIT-ROOSEVELT,0.0,0.0,0.0,4
206993,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/02/2019,13:00:00,REGULAR,5554,386,2019-08-02 13:00:00,TRAM2-R469-00-05-01-RIT-ROOSEVELT,0.0,0.0,0.0,4
206994,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,08/02/2019,17:00:00,REGULAR,5554,386,2019-08-02 17:00:00,TRAM2-R469-00-05-01-RIT-ROOSEVELT,0.0,0.0,0.0,4


In [42]:
new = no_outliers.sort_values('entries_diff',ascending=False)

new[['turnstiles', 'STATION', 'DATE', 'DESC', 'TIME', 'day_of_week','turnstile_turns']].head(50).sort_values('turnstiles')
                                                                                                             

,turnstiles,STATION,DATE,DESC,TIME,day_of_week,turnstile_turns
1425,A010-R080-00-00-07-57 ST-7 AV,57 ST-7 AV,07/31/2019,REGULAR,20:00:00,2,2.314000e+03
1419,A010-R080-00-00-07-57 ST-7 AV,57 ST-7 AV,07/30/2019,REGULAR,20:00:00,1,2.308000e+03
1413,A010-R080-00-00-07-57 ST-7 AV,57 ST-7 AV,07/29/2019,REGULAR,20:00:00,0,2.257000e+03
1641,A011-R080-01-00-04-57 ST-7 AV,57 ST-7 AV,08/01/2019,REGULAR,20:00:00,3,2.518000e+03
1629,A011-R080-01-00-04-57 ST-7 AV,57 ST-7 AV,07/30/2019,REGULAR,20:00:00,1,2.365000e+03
1635,A011-R080-01-00-04-57 ST-7 AV,57 ST-7 AV,07/31/2019,REGULAR,20:00:00,2,2.393000e+03
2859,A022-R022-01-00-00-34 ST-HERALD SQ,34 ST-HERALD SQ,08/01/2019,REGULAR,20:00:00,3,3.173000e+03
2865,A022-R022-01-00-00-34 ST-HERALD SQ,34 ST-HERALD SQ,08/02/2019,REGULAR,20:00:00,4,3.040000e+03
2847,A022-R022-01-00-00-34 ST-HERALD SQ,34 ST-HERALD SQ,07/30/2019,REGULAR,20:00:00,1,3.187000e+03
2841,A022-R022-01-00-00-34 ST-HERALD SQ,34 ST-HERALD SQ,07/29/2019,REGULAR,20:00:00,0,2.741000e+03


In [64]:
station_df = no_outliers.groupby('STATION')
station_totals=station_df['turnstile_turns'].sum()

station_totals.sort_values(ascending=False).head(30)

STATION
HUNTS POINT AV     3.083585e+09
GRD CNTRL-42 ST    6.039917e+06
66 ST-LINCOLN      2.093391e+06
34 ST-PENN STA     1.898187e+06
34 ST-HERALD SQ    1.400539e+06
TIMES SQ-42 ST     1.196956e+06
23 ST              1.126268e+06
14 ST-UNION SQ     1.126184e+06
FULTON ST          1.074935e+06
42 ST-PORT AUTH    1.033920e+06
86 ST              9.170790e+05
MORGAN AV          8.788540e+05
125 ST             8.398200e+05
CANAL ST           8.030450e+05
59 ST COLUMBUS     7.660630e+05
47-50 STS ROCK     7.274570e+05
96 ST              7.102940e+05
59 ST              6.815550e+05
FLUSHING-MAIN      6.543050e+05
14 ST              6.229060e+05
PATH NEW WTC       6.013240e+05
CHAMBERS ST        5.391640e+05
50 ST              5.347020e+05
42 ST-BRYANT PK    5.282780e+05
72 ST              5.269290e+05
JKSN HT-ROOSVLT    5.256410e+05
28 ST              5.196520e+05
W 4 ST-WASH SQ     5.000020e+05
ATL AV-BARCLAY     4.767180e+05
WALL ST            4.639800e+05
Name: turnstile_turns, dtype: fl

In [16]:
import datetime as dt
df_sorted['day_of_week']=df_sorted.converted_time.dt.dayofweek

In [56]:
def reverseString(word):
    reversed = word[::-1]
    reversed_word = ""
    
    for i in reversed:
        reversed_word += reversed[i]
        
    return reversed_word

In [57]:
x = reverseString("hello")
print(x)

TypeError: string indices must be integers

In [60]:
# Question 1 - Drew H, Albert L, Nick W 

def reverse_String(s):
    l = [char for char in s]
    reverse = l[::-1]
    
    return "".join(reverse)

# Question 2 - Drew H, Albert L, Nick W



In [61]:
reverse_String("hello")

'olleh'

In [ ]:
def CanYouSpell(s,word):
    for letter in word:
        if letter in s:
            return True
        if 